In [1]:
import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.regularizers import *
from tensorflow.keras.constraints import unit_norm
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import Sequence
import tensorflow.keras.backend as K
import pandas as pd
import os
import numpy as np
from tqdm import tqdm
import time

In [5]:
checkpoint_path='/nas/home/cborrelli/tripletloss_bot/checkpoints/vggish/vggish_model.ckpt'


In [6]:
def get_vggish(input_shape):
    img_input = Input(shape=input_shape)
    # Block 1
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='conv1')(img_input)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='pool1')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='pool2')(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='conv3_1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='conv3_2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='pool3')(x)

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='conv4_1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='conv4_2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='pool4')(x)

    # Block fc
    x = Flatten(name='flatten')(x)
    x = Dense(4096, activation='relu', name='fc1_1')(x)
    x = Dense(4096, activation='relu', name='fc1_2')(x)
    x = Lambda(lambda x: tf.math.l2_normalize(x, axis=1))(x)
    
    vggish = Model(img_input, x, name='vggish')
    
    return vggish
    

In [7]:
def get_vggish_tripletloss(input_shape):
    img_input = Input(shape=input_shape)
    label_input = Input(shape=(1,))

    base_model = get_vggish(input_shape)
    
    # Initialize base model with VGGish weights
    reader = tf.compat.v1.train.NewCheckpointReader(checkpoint_path)
    var_to_shape_map = reader.get_variable_to_shape_map()
    
    tensor_layers_list = []
    for key in var_to_shape_map:
        tensor_layers_list.append('/'.join(key.split('/')[:-1]))

    for index, t in enumerate(tensor_layers_list):
        weights_key = t + '/weights'
        bias_key = t + '/biases'
        weights = reader.get_tensor(weights_key)
        biases = reader.get_tensor(bias_key)

        keras_layer_name = t.split('/')[-1]
        if keras_layer_name == 'logits' or keras_layer_name == 'fc2':
            continue

        base_model.get_layer(keras_layer_name).set_weights([weights, biases])
    
    embeddings = base_model(img_input)

    labels_plus_embeddings = concatenate[(label_input, embeddings)]

    model = Model(inputs=[img_input, label_input], outputs=labels_plus_embeddings)
    return model

In [8]:
prova = get_vggish_tripletloss(input_shape=(96, 64, 1))

TypeError: 'function' object is not subscriptable